In [13]:
import pandas as pd
import os 
from dotenv import load_dotenv
from tqdm import tqdm

load_dotenv()

True

In [2]:
from src.evaluation.qa_evaluator import QAEvaluator
from src.rag.rag import RAG

/Users/kirill/rag_tg_2025/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/kirill/rag_tg_2025/src/config.py:40: UserWarning: Qdrant client version 1.16.2 is incompatible with server version 1.14.1. Major versions should match and minor version difference must not exceed 1. Set check_compatibility=False to skip version check.
  qdrant_client = QdrantClient(url=QDRANT_URL)


In [3]:
rag = RAG(
    embed_model_name="deepvk/USER-bge-m3",
    embed_index_name="recursive_USER-bge-m3",
)

In [5]:
test_cases = pd.read_csv('./test_cases.csv')

In [7]:
test_cases.head()

,message_id,original_text,strict_question,real_question
0,130738,Итальянский суд принял решение экстрадировать ...,Какое решение приняло итальянское судопроизвод...,Что там с Кузнецовым — его в Германию выдадут ...
1,129361,Пять пассажиров автобуса №793 пострадали в ДТП...,Сколько пассажиров автобуса №793 пострадали в ...,"Сколько человек в автобусе 793 пострадали, ког..."
2,133468,Владимир Путин утвердил концепцию государствен...,Кто утвердил концепцию государственной миграци...,Кто там утвердил новую миграционную концепцию ...
3,123139,Генпрокуратура и Минюст подали в Верховный суд...,Какое юридическое действие предприняли Генерал...,Что Генпрокуратура и Минюст сделали с сатанист...
4,129894,Обломки дрона обнаружили польские пограничники...,Где и кем был обнаружен непилотируемый летател...,Что там польские пограничники нашли рядом с Бе...


In [9]:
evaluator = QAEvaluator(
    df=test_cases,
    text_column="original_text",
    temperature=0.0,
    api_key=os.getenv("OPENROUTER_API_KEY")
)


In [10]:
batch_size = 16
question_iterator = evaluator.get_questions(
    question_column="strict_question",  # или "real_question"
    batch_size=batch_size
)

print(f"Всего батчей: {len(question_iterator)}")

Всего батчей: 11


In [14]:
generated_answers = []

for query in tqdm(test_cases["real_question"]):
    results = rag.invoke(query)
    generated_answers.append(
        (results["answer"], results["reason"])
    )

100%|█████████████████████████████████████████| 167/167 [03:36<00:00,  1.30s/it]


In [19]:
test_cases["generated_answer"] = [p[0] for p in generated_answers]
test_cases["generated_reason"] = [p[1] for p in generated_answers]
test_cases.head()

,message_id,original_text,strict_question,real_question,generated_answer,generated_reason
0,130738,Итальянский суд принял решение экстрадировать ...,Какое решение приняло итальянское судопроизвод...,Что там с Кузнецовым — его в Германию выдадут ...,Не знаю.,В контексте нет однозначного ответа на вопрос ...
1,129361,Пять пассажиров автобуса №793 пострадали в ДТП...,Сколько пассажиров автобуса №793 пострадали в ...,"Сколько человек в автобусе 793 пострадали, ког...",5,"В первом абзаце контекста указано, что пять па..."
2,133468,Владимир Путин утвердил концепцию государствен...,Кто утвердил концепцию государственной миграци...,Кто там утвердил новую миграционную концепцию ...,Владимир Путин,"В первом предложении контекста указано, что Вл..."
3,123139,Генпрокуратура и Минюст подали в Верховный суд...,Какое юридическое действие предприняли Генерал...,Что Генпрокуратура и Минюст сделали с сатанист...,Генпрокуратура и Минюст подали в Верховный суд...,"Это указано в первом абзаце контекста, где гов..."
4,129894,Обломки дрона обнаружили польские пограничники...,Где и кем был обнаружен непилотируемый летател...,Что там польские пограничники нашли рядом с Бе...,Да,"В контексте указано, что обломки дрона обнаруж..."


In [21]:
answers = test_cases["generated_answer"]
metrics = evaluator.evaluate_answers(answers, show_progress=True)

Оценка ответов:  54%|██████████████            | 90/167 [03:42<15:54, 12.39s/it]

Ошибка при оценке ответа: Failed to parse AnswerEvaluation from completion {"is_valid": true, "relevance_score": 1.0}. Got: 2 validation errors for AnswerEvaluation
completeness_score
  Field required [type=missing, input_value={'is_valid': True, 'relevance_score': 1.0}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
factual_accuracy_score
  Field required [type=missing, input_value={'is_valid': True, 'relevance_score': 1.0}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 


Оценка ответов: 100%|█████████████████████████| 167/167 [05:25<00:00,  1.95s/it]


In [29]:
metrics_df = pd.DataFrame(
    data=[(
        metrics["total_questions"], metrics["valid_answers"], metrics["accuracy"],\
        metrics["avg_relevance"], metrics["avg_completeness"], metrics["avg_factual_accuracy"], \
        metrics["combined_score"]
          )],
    columns=["total_questions", "valid_answers", "accuracy", "avg_relevance", "avg_completeness", \
            "avg_factual_accuracy", "combined_score"]
)
metrics_df

,total_questions,valid_answers,accuracy,avg_relevance,avg_completeness,avg_factual_accuracy,combined_score
0,167,114,0.682635,0.819162,0.57485,0.81976,0.737924


In [32]:
pd.DataFrame(metrics["detailed_results"]).sample(5)

,index,question,answer,is_valid,relevance_score,completeness_score,factual_accuracy_score
112,112,На каком уровне оказалась цена нефти марки Ura...,"Да, цена российской нефти марки Urals в Новоро...",True,1.0,0.5,1.0
37,37,"Согласно официальному сообщению Росавиации, ка...",В аэропорту Ярославля введены временные ограни...,True,1.0,1.0,1.0
140,140,По какому инциденту Следственный комитет Росси...,Следственный комитет завел уголовное дело по ф...,True,1.0,1.0,1.0
0,0,Какое решение приняло итальянское судопроизвод...,Не знаю.,False,0.5,0.0,0.0
98,98,"Что произошло на фестивале в Пенсильвании, США...",На фестивале в Пенсильвании минивэн въехал в т...,True,1.0,1.0,1.0
